In [1]:
from agential.cog.functional.critic import _prompt_agent, _prompt_critic
from agential.cog.prompts.critic import (
    CRITIC_INSTRUCTION_HOTPOTQA, 
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
)
from langchain.prompts import PromptTemplate
import ast

In [2]:
from langchain_community.chat_models.openai import ChatOpenAI

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

answer = _prompt_agent(
    llm=llm, 
    keys={
        "question": question, 
        "examples": HOTPOTQA_FEWSHOT_EXAMPLES_COT
    }, 
    prompt_template=CRITIC_INSTRUCTION_HOTPOTQA
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


<PROMPT IN =======================================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profes

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<PROMPT OUT =======================================================================>
Let's break it down step by step:
1. The person was once considered the best kickboxer in the world.
2. They have been involved in controversies related to unsportsmanlike conduct in the sport.
3. They have also been involved in crimes of violence outside of the ring.
Based on these clues, the person in question is likely Badr Hari, a Dutch-Moroccan kickboxer known for his skills in the ring but also for his controversial behavior and legal issues. So the answer is: Badr Hari.
<PROMPT OUT =======================================================================>


In [3]:
print(answer)

Let's break it down step by step:
1. The person was once considered the best kickboxer in the world.
2. They have been involved in controversies related to unsportsmanlike conduct in the sport.
3. They have also been involved in crimes of violence outside of the ring.
Based on these clues, the person in question is likely Badr Hari, a Dutch-Moroccan kickboxer known for his skills in the ring but also for his controversial behavior and legal issues. So the answer is: Badr Hari.


In [4]:
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.tools.google_search.tool import GoogleSearchResults, GoogleSearchRun

search = GoogleSearchResults(api_wrapper=GoogleSearchAPIWrapper(), num_results=5)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(
c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchResults` was deprecated in LangChain 0.0.33 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchResults`.
  warn_deprecated(


In [5]:
template = """{examples}
(END OF EXAMPLES)

Question: {question}
Proposed Answer: {answer}

What's the problem with the above answer?

1. Plausibility:

{critique}

{format_instructions}"""

In [6]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="answer_type", description="Can either be SEARCH (search Google for more info) or ANSWER (critique answers the question)."),
    ResponseSchema(
        name="value",
        description="Can either be the search query (if answer_type=SEARCH) or the answer (if answer_type=ANSWER)",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=template,
    input_variables=["examples", "question", "answer", "critique"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

a = chain.invoke({
    "examples": HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    "question": question,
    "answer": "This controversial boxer is Andrew Tate.",
    "critique": ""
})

In [7]:
search(a['value'])

'[{\'title\': \'Why do people act like Andrew Tate was an elite level kickboxer ...\', \'link\': \'https://www.reddit.com/r/Kickboxing/comments/142z5fn/why_do_people_act_like_andrew_tate_was_an_elite/\', \'snippet\': "Jun 7, 2023 ... No, but you don\'t have to be in the top 1% to be considered ... highest level in Glory kickboxing and he\'s a 6 time world champion. ... one of the\\xa0..."}, {\'title\': \'Badr Hari - Wikipedia\', \'link\': \'https://en.wikipedia.org/wiki/Badr_Hari\', \'snippet\': \'Known for his aggressive and explosive fighting style, Hari is widely considered one of the greatest kickboxers of all time.\'}, {\'title\': \'Was Andrew Tate a GOOD Kickboxer? : r/martialarts\', \'link\': \'https://www.reddit.com/r/martialarts/comments/wjm5f3/was_andrew_tate_a_good_kickboxer/\', \'snippet\': "Aug 8, 2022 ... ... best kickboxer for his weight class, becoming kickboxing champion 4 times. ... He fought on K1 once. And ... ISKA isn\'t a top shelf or world class\\xa0..."}, {\'tit

In [8]:
if a['answer_type'] == "SEARCH":
    query = a['value']
    search_results = ast.literal_eval(search(a['value']))
elif a['answer_type'] == "ANSWER": 
    answer = a['value']
else:
    pass

In [20]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools.google_serper.tool import GoogleSerperRun 

search_serper = GoogleSerperRun(api_wrapper=GoogleSerperAPIWrapper())

In [21]:
serper_result = search_serper.run(query)